### Подготовка

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2
!pip install pytorch-lightning --upgrade
## Install NeMo
BRANCH = 'r1.17.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
from IPython.display import clear_output
clear_output()

In [ ]:
import nemo
# импортируе Nemo коллекцию, которая содержит все asr модели
import nemo.collections.asr as nemo_asr

[NeMo W 2023-04-25 14:00:08 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-25 14:00:11 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [ ]:
# import torch
# device = torch.device('cuda')

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget
import wave
import contextlib
from google.colab import files
import pandas as pd
import json
import uuid
from IPython.display import clear_output

Приписываем пути к манифестам

In [ ]:
data_dir = '/content/drive/MyDrive/vkr'
manifest_test_filepath = '/content/drive/MyDrive/vkr/test_golos/test_golos.jsonl'
manifest_train_filepath = '/content/drive/MyDrive/vkr/train_golos/train_golos.jsonl'
test_data_abs_path = '/content/drive/MyDrive/vkr/test_golos/test_golos.jsonl'
train_data_abs_path ='/content/drive/MyDrive/vkr/train_golos/train_golos.jsonl'

Преобразуем стерео аудио в моно аудио

In [ ]:
# import os
# import subprocess

# def convert_to_wav(directory_path):
#     for root, dirs, files in os.walk(directory_path):
#         for file in files:
#             if file.endswith('.wav'):
#                 input_path = os.path.join(root, file)
#                 output_path = os.path.splitext(input_path)[0] + '3.wav'
#                 subprocess.call(['ffmpeg', '-i', input_path, '-vn', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', output_path])
#                 # os.remove(input_path)

In [ ]:
# convert_to_wav(train_data_abs_path)

In [ ]:
from pydub import AudioSegment

def get_audio_duration(filepath):
    audio = AudioSegment.from_file(filepath)
    duration = audio.duration_seconds
    return duration

In [ ]:
def read_json_file(file):
    with open(file, "r") as r:
        response = r.read()
        response = response.replace('\n', '')
        response = response.replace('}{', '},{')
        response = "[" + response + "]"
        return json.loads(response)

In [ ]:
import os
import json

def make_manifest(root_dir, manifest_path):
    manifest = []
    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if os.path.isdir(class_path):
            for audio_file in os.listdir(class_path):
                audio_path = os.path.join(class_path, audio_file)
                if audio_path.endswith('.wav'):
                    duration = get_audio_duration(audio_path)
                    manifest.append({
                        'audio_filepath': audio_path,
                        'duration': duration,
                        'text': class_dir,
                    })

    with open(manifest_path, 'w') as f:
        for line in manifest:
            json.dump(line, f)
            f.write('\n')

Можно построить манифесты заново

In [ ]:
# audio_dir = '/content/drive/MyDrive/vkr/test_golos/crowd/tmp'
# manifest_path = '/content/drive/MyDrive/vkr/test_golos/crowd/tmp/tmp_manifest.json'

# make_manifest(audio_dir, manifest_path)

In [ ]:

# audio_dir = test_data_abs_path
# manifest_path = manifest_test_filepath

# make_manifest(audio_dir, manifest_path)

### Обучение

Подгружаем файлы, которые мы добавим к медицинским терминам

In [ ]:
def load(golos_url, fname):
  if not os.path.exists(os.path.join(data_dir, fname)):
      golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
      print(f"Dataset downloaded at: {golos_path}")
  else:
      print("Tarfile already exists.")
      golos_path = os.path.join(data_dir, fname)
  return golos_path

In [ ]:
! head /content/drive/MyDrive/vkr/train_golos/crowd/manifest_train_golos.jsonl

{"id": "77b380796d242cf5bc09cfb551cffecd", "audio_filepath": "crowd/0/77b380796d242cf5bc09cfb551cffecd.wav", "text": "\u0430\u043b\u0438\u0431\u0438", "duration": 1.21}
{"id": "404e1eecbc38a687e036739f2e5dd6d3", "audio_filepath": "crowd/0/404e1eecbc38a687e036739f2e5dd6d3.wav", "text": "\u0441\u043a\u0430\u0437\u043a\u0430", "duration": 2.9}
{"id": "f913db484595c1bcdd9476194b08a1f4", "audio_filepath": "crowd/0/f913db484595c1bcdd9476194b08a1f4.wav", "text": "\u043d\u0430\u0439\u0434\u0438 \u0432 \u043d\u0435\u0442\u0435 \u043c\u0443\u043b\u044c\u0442 \u0441\u0442\u0430\u043b\u044c\u043d\u043e\u0439 \u0433\u0438\u0433\u0430\u043d\u0442", "duration": 4.76}
{"id": "eca448acf1712fa7ca08aede3d2df1ac", "audio_filepath": "crowd/0/eca448acf1712fa7ca08aede3d2df1ac.wav", "text": "\u043c\u043e\u043d\u0430 \u043b\u0438\u0437\u0430", "duration": 2.3}
{"id": "a366eecb0a85ef76463e1f4b5188b698", "audio_filepath": "crowd/0/a366eecb0a85ef76463e1f4b5188b698.wav", "text": "\u0441\u043c\u043e\u0442\u0440\u04

In [ ]:
! head /content/drive/MyDrive/vkr/test_golos/crowd/manifest_test_golos.jsonl

{"id": "e632f7d39c15e7edfc665b91e6f2071f", "audio_filepath": "files/e632f7d39c15e7edfc665b91e6f2071f.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0432\u043e\u0441\u043f\u0440\u043e\u0438\u0437\u0432\u0435\u0434\u0438 \u043c\u0443\u0437\u044b\u043a\u0443 \u0432\u043f\u0435\u0440\u0435\u043c\u0435\u0448\u043a\u0443", "duration": 4.9}
{"id": "5db5df8bb9e3b6660b2a04b34d4a355d", "audio_filepath": "files/5db5df8bb9e3b6660b2a04b34d4a355d.wav", "text": "\u043d\u0430\u0439\u0442\u0438 \u0441\u0435\u0440\u0438\u0430\u043b \u0433\u0440\u0438\u0433\u043e\u0440\u0438\u0439 \u0440", "duration": 3.652}
{"id": "2c471aedc6979109f28cd53c58f8c4fb", "audio_filepath": "files/2c471aedc6979109f28cd53c58f8c4fb.wav", "text": "\u043f\u0440\u044f\u043c\u043e\u0439 \u044d\u0444\u0438\u0440 \u0430\u043f\u043b \u043c\u0430\u043d\u0447\u0435\u0441\u0442\u0435\u0440 \u044e\u043d\u0430\u0439\u0442\u0435\u0434 \u0442\u043e\u0442\u0442\u0435\u043d\u0445\u044d\u043c", "duration": 4.34175}
{"id": "756a137ee9debde4a008a

In [ ]:
import json
def build_manifest(manifest_rel, manifest_abs, test=False):
    n = 0
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    with open(manifest_rel, 'r') as fin:
        with open(manifest_abs, 'w') as fout:
            for line in fin:
                n += 1
                if n < 20000 and not test:
                  continue
                if n > 50000:
                  break
                metadata = json.loads(line)
                if metadata['id'] == '375e547e51a71f9e18b811dd89244baf':
                  # print('empty')
                  continue
                metadata["audio_filepath"]=os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, fout)
                fout.write('\n')

Строим для них манифесты

In [ ]:
# # Building Manifests
# print("******")
# train_rel = '/content/drive/MyDrive/vkr/train_golos/crowd/manifest_train_golos.jsonl'
# train_abs = '/content/drive/MyDrive/vkr/train_golos/crowd/train_golos.jsonl'
# if not os.path.isfile(train_abs):
#   build_manifest(train_rel, train_abs, False)
# train_manifest = train_abs
# print("train_manifest", train_manifest)

# test_rel = '/content/drive/MyDrive/vkr/test_golos/crowd/manifest_test_golos.jsonl'
# test_abs = '/content/drive/MyDrive/vkr/test_golos/crowd/test_golos.jsonl'
# if not os.path.isfile(test_abs):
#   build_manifest(test_rel, test_abs, True)
# test_manifest = test_abs
# print("test_manifest", test_manifest)

Импортируем модель

In [ ]:
model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

[NeMo W 2023-04-25 15:14:44 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2023-04-25 15:14:44 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2023-04-25 15:14:44 features:287] PADDING: 16
[NeMo I 2023-04-25 15:14:45 save_restore_connector:247] Model EncDecCTCModel was successfully restored from /content/drive/MyDrive/vkr/QuartzNet15x5_golos.nemo.


Настраиваем параметры модели

In [ ]:
import copy
from omegaconf import DictConfig
test_manifest = '/content/drive/MyDrive/vkr/test_golos/crowd/test_golos.jsonl'
train_manifest = '/content/drive/MyDrive/vkr/train_golos/crowd/train_golos.jsonl'
batch = 8

params_model = model._cfg
params_model['train_ds']['parser'] = 'base'
params_model['validation_ds']['parser'] = 'base'
params_model['test_ds']['parser'] = 'base'

params_model['train_ds']['manifest_filepath'] = train_manifest
params_model['train_ds']['batch_size'] = batch

params_model['test_ds']['manifest_filepath'] = test_manifest
params_model['test_ds']['batch_size'] = batch

params_model['validation_ds']['manifest_filepath'] = test_manifest
params_model['validation_ds']['batch_size'] = batch

In [ ]:
model.setup_training_data(train_data_config=params_model['train_ds'])
model.setup_test_data(test_data_config=params_model['test_ds'])
model.setup_validation_data(val_data_config=params_model['validation_ds'])

lr = 0.0005

params_optim = copy.deepcopy(params_model['optim'])
params_optim['lr'] = lr
model.setup_optimization(optim_config=DictConfig(params_optim))

[NeMo I 2023-04-25 15:14:56 collections:193] Dataset loaded with 13115 files totalling 9.28 hours
[NeMo I 2023-04-25 15:14:56 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-25 15:14:57 collections:193] Dataset loaded with 3555 files totalling 3.99 hours
[NeMo I 2023-04-25 15:14:57 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-25 15:14:58 collections:193] Dataset loaded with 3555 files totalling 3.99 hours
[NeMo I 2023-04-25 15:14:58 collections:194] 0 files were filtered totalling 0.00 hours


[NeMo W 2023-04-25 15:14:58 modelPT:611] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2023-04-25 15:14:58 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )


[NeMo W 2023-04-25 15:14:58 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     lr: 0.0005
     weight_decay: 0.001
 ),
 None)

Посмотрим на качество модели до дообучения

In [ ]:
# перекидываем модель на GPU
model.cuda()

# Оценивать качесвто модели будем по метрике Word Error Rate=numerator/denumerator
wer_nums = []
wer_denoms = []
# Тут у нас проходит итерация по всем естовым батчам, которые содержат следующие компоненты
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
for test_batch in model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = model._wer.compute()
        model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())
        # освобождаем GPU память
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions
# Считаем метрику
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")
print(f'WER до дообучения составляет {sum(wer_nums)/sum(wer_denoms)}')

WER = 0.19798002343619217
WER до дообучения составляет 0.19798002343619217


Запускаем обучение модели на наших данных

In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=5, precision=16)

[NeMo W 2023-04-25 14:28:35 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
      rank_zero_deprecation(
    
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-04-25 14:28:36 modelPT:722] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0005
        weight_decay: 0.001
    )
[NeMo I 2023-04-25 14:28:36 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7feceacdd370>" 
    will be used during training (effective maximum steps = 8200) - 
    Parameters : 
    (warmup_steps: 500
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 8200
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
37.859    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
model.save_to('content/drive/MyDrive/vkr/QuartzNet15x5_golos.nemo')

In [ ]:
# перекидываем модель на GPU
model.cuda()

# Оценивать качесвто модели будем по метрике Word Error Rate=numerator/denumerator
wer_nums = []
wer_denoms = []
# Тут у нас проходит итерация по всем естовым батчам, которые содержат следующие компоненты
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
for test_batch in model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = model._wer.compute()
        model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())
        # освобождаем GPU память
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions
# Считаем метрику
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")
print(f'WER после дообучения составляет {sum(wer_nums)/sum(wer_denoms)}')

WER = 0.10473745884716254
WER после дообучения составляет 0.10473745884716254
